In [2]:
# パス設定
import sys
sys.path.append('/home/is/akiyoshi-n/my-project')

In [3]:
import os
# 使用するGPUを指定. この環境変数の場所は，pytorchをimportする前に入れる
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from pathlib import Path
from datetime import datetime
from src.my_project.dataset import load_dataset_2class_classification, split_test_data, load_text_dataset, load_dataset_2class_classification_v2, split_test_data_stratify, preprocess_for_Trainer, load_sequential_emotion_data
from src.my_project.train_v2 import ActClassifier
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
import wandb
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [4]:
DATASET_PATH = Path('/home/is/akiyoshi-n/my-project/data')
# 本日の日付
timestamp = datetime.now().strftime("%Y-%m-%d")
# 出力先ディレクトリ
output_dir = Path('/home/is/akiyoshi-n/my-project/outputs/{}'.format(timestamp))

In [4]:
# 最大トークン数
MAX_LEN = 128
# バッチサイズ
BATCH_SIZE = 16
# エポック数
NUM_EPOCHS = 100
# 学習率
LEARNING_RATE = 2e-5
# Cross Validation時のFold数
NUM_FOLDS = 5
# 早期停止のための忍耐値
PATIENCE = 2
# 乱数シード
SEED = 2023
# クラス数
NUM_LABELS = 2

In [5]:
# 辞書型でデータ取得
data = load_dataset_2class_classification(f"{DATASET_PATH}/act_classification_final_ChatGPT4.xlsx")

In [6]:
# 東北大BERT-v3
MODEL_NAME = 'cl-tohoku/bert-base-japanese-v3'
Classifier_model = ActClassifier(model_name = MODEL_NAME, num_labels=2, seed=SEED)

In [7]:
# testデータと訓練に使用するデータに分割
dataset, eval_dataset, a, b = split_test_data_stratify(data=data, test_size=0.2, SEED=SEED)

In [8]:
model = Classifier_model.train_model(dataset, eval_dataset, MAX_LEN, NUM_EPOCHS, LEARNING_RATE, BATCH_SIZE, PATIENCE, output_dir, project_name='ActClassification_2class_5_7', run_name='basic_model')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Parameter 'fn_kwargs'={'tokenizer': BertJapaneseTokenizer(name_or_path='cl-tohoku/bert-base-japanese-v3', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False,

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.691300,0.637914,0.658333,0.657953
2,0.622900,0.586818,0.695833,0.695701
3,0.518200,0.565318,0.691667,0.691474
4,0.376100,0.637142,0.675000,0.671717
5,0.206200,0.721510,0.725000,0.724311


In [9]:
lifestory = load_sequential_emotion_data(f"{DATASET_PATH}/downloaded_files/LifeStory_2023_season2.xlsx")

In [10]:
lifestory

{'texts': ['オークションで狙っていたものが買えなかった',
  '腰痛が悪化した',
  'GW中は金欠気味だったので友人の誘いを断っていたのが悲しかった',
  'むかしから見ていた人気ストリーマーが若くして亡くなった',
  '大切なペットが亡くなってしまったことがあります。長年一緒に生活してきたパートナーのような存在であり、家族として愛されていたペットがいなくなるという現実に直面することは、非常につらいものでした。',
  '能登で地震',
  'GW中に子供と巨人戦を観戦しにいったが、巨人が負けてしまったこと。',
  '友達の飼っていた猫が死んだ',
  '昔の悲しい出来事を夢で見て、当時の事が思い出されて悲しさが蘇ってきた。',
  'ゲーム機の裏蓋が壊れた。',
  '嬉しい',
  '外出して食事をするにもお金が無く、好きなものを食べられなく悲しい',
  '1か月近く前に、家の中で転んで頭部を床に直撃して以来、めまいや頭痛で体調がずっと悪くとても悲しい。',
  '自宅周辺で異臭がする',
  '政治家が自分たちの利権しか考えておらず、悲しい。',
  '歯槽膿漏で歯が一本抜けてしまった',
  '人生を共にしてきた先輩と些細なことで仲たがいして、音信沸あになったこと。',
  '地震',
  'なし',
  '旅行先でいろいろ食べたかったのにパンパンで食べられなかった',
  '入院していた父が突然亡くなった。',
  '大事に育てていた牡丹の花が咲く前に虫に食われて折れていたことです',
  'おやしんだ',
  '犬の糞の不始末',
  '3年前の病気(心臓病)によりコロナ禍で、普通の人より自粛をしいられたこと',
  '家族とケンカした',
  '自衛隊ヘリ墜落事故',
  '仕事',
  '対人関係',
  '半年前、帯状疱疹にかかりすぐに治療したが神経痛が残り、今もつらい思いをしてる。',
  '２０年超勤務した会社が倒産し、その後未だに定職に就けず困っている',
  '貸し農園で栽培していた極早生の玉ねぎが\u3000バカ見たいに大収穫ができた',
  '妻が私に内緒で借金をし、支払えなくなって家出した時は本当に悲しかった。',
  '主人を亡くした',
  '最近、車をぶつけてしまった',
  '派遣の仕事に応募したら、

In [11]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
test_data = preprocess_for_Trainer(lifestory, tokenizer, MAX_LEN)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

In [12]:
logits = model.predict(test_data).predictions

In [13]:
import numpy as np
predictions = np.argmax(logits, axis=-1) # 最大値のラベルを予測値とする

In [14]:
predictions[:20]

array([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1])

In [15]:
predictions.sum()

3677

In [22]:
# 予測値が1の部分のみを抽出
selected_texts = {'texts': [lifestory['texts'][i] for i in range(len(lifestory['predictions'])) if lifestory['predictions'][i] == 1]}

In [23]:
# selected_textsの'textの部分が20文字より大きく100文字未満のものを抽出
a = [i for i in selected_texts['texts'] if len(i) > 20 and len(i) < 100]
selected_texts_new = {'texts': a}

In [25]:
len(selected_texts_new['texts'])

1314

In [27]:
# # 辞書をpickleで保存
# import pickle
# with open(f"{DATASET_PATH}/selected_texts_2023_season2.pkl", 'wb') as f:
#     pickle.dump(selected_texts_new, f)

In [12]:
import pickle
# 保存したpickleファイルを読み込む
with open(f"{DATASET_PATH}/selected_texts_lifestory/selected_texts_2023_season1.pkl", 'rb') as f:
    a = pickle.load(f)

In [13]:
a

{'texts': ['北海道済みなので、仕方ないことだがせっかく綺麗に除雪したのに翌日には大吹雪で、また雪が積もっていたのが悲しかった',
  '買った商品が翌日特価でさらに安くなっていたこと。',
  '洗剤がなくなり安いと思いネットで購入したが、近くのドラッグストアのほうがもっと安かったこと。',
  '突然、一局だけチャンネルが時間帯によって映ったり映らなかったりして好きな番組がリアルタイムに見れなくて悲しい。',
  '外出先で手袋を片方なくし探したが見つからなかった。',
  '昨年末のエリザベス女王とヴィヴィアンウエストウッドさんの死',
  'スマホを落として塗装が剥げてしまったこと。',
  '少額のポイントしかもらえないタスクしかなくて悲しい',
  '片目が見えにくくて手術をした結果、網膜剝離を起こして、車の運転がつらくなった。',
  '誕生日にケーキを貰ったが忙しくて、食べる前に賞味期限が切れてしまった。',
  '老舗デパートの終了##懐かしさもあり、寂しさもあります。',
  '飲食店でもらったスクラッチクーポンがはずれだった',
  '煮物を煮ているときに焦してしまって悲しかった',
  '１０年以上飼っていた飼い犬が死んだ時が悲しかった',
  '持病が日によって症状の度合いが違うが、最近は悪い日が凄く多いこと。早く安定して欲しい。',
  '最も楽しみにしていた姪っ子お手製のハンバーガーが思いっきりしょっぱくて悲しい。',
  '気に入っていた腕時計が壊れて動かなくなったこと',
  '知り合いが昨年末にコロナウイルスに罹患し、自宅療養中でしたが、体調が急に悪くなり、救急車を呼びましたが、病院でお亡くなりになリました。',
  '子どもに辛く当たってしまった。部屋を片さないことを一言叱ればいいだけなのにぐだぐだと嫌味を言って嫌がらせをしてしまった。',
  '買ったばかりの日本酒をグラスに注ごうとして、こぼした',
  '大事にしてたアイスを子供に食べられて悲しい',
  '車の車検で指摘された修理箇所か多く、費用が予想していたよりもかかった。',
  '現在より待遇のよい職場に応募するが専攻から落選して悲しいです。',
  'そばを食べにわざわざ遠方から来たのに、営業時間が終わっていて悲しい。',
  'コロナになり、遊ぶ予定を

In [11]:
len(a['texts'])

989

In [10]:
len(b['texts'])

1314

### LLMでアノテーション

In [6]:
from ollama import Client

In [7]:
def get_response(message, prompt_message):
    contents = prompt_message + "\n「" + message + "」\n### 応答\n"
    response = client.chat(
        # model="command-r",
        model = 'command-r-plus',
        messages=[
            {
                "role": "user",
                "content": contents,
            },
        ],
    )
    return response["message"]["content"]

In [8]:
import tqdm

In [9]:
response = get_response(selected_texts['texts'][0], prompt_message)

NameError: name 'prompt_message' is not defined

In [9]:
client = Client(host='http://benihi.naist.jp:11434/')
results = []
# prompt_message.txtファイルから文字列を読み取る
with open(f"{DATASET_PATH}/prompt_message_v3.txt", 'r') as f:
    prompt_message = f.read()

count = 0

for text in tqdm.tqdm(selected_texts['texts']):
    response = get_response(text, prompt_message)
        # 抽出された行動を表示
    # 辞書として結果を追加
    results.append({'text': text, 'response': response})
    count += 1
    if count == 50:
        break

  2%|▏         | 49/2674 [02:09<1:55:41,  2.64s/it]


In [11]:
results

[{'text': 'スマホ落とした', 'response': '4. 行動なし'},
 {'text': '北海道済みなので、仕方ないことだがせっかく綺麗に除雪したのに翌日には大吹雪で、また雪が積もっていたのが悲しかった',
  'response': '4. 行動なし'},
 {'text': '買った商品が翌日特価でさらに安くなっていたこと。', 'response': '3-B買い物\n\n'},
 {'text': 'お金がない', 'response': '4. 行動なし'},
 {'text': '洗剤がなくなり安いと思いネットで購入したが、近くのドラッグストアのほうがもっと安かったこと。',
  'response': '3-B.買い物，1-C-b.食事（飲酒なし・不明）'},
 {'text': '親戚が亡くなった', 'response': '3-D-a:交際‐物理的接触，1-C-b：食事－飲酒なし・不明'},
 {'text': '家族から無視されている。', 'response': '4. 行動なし'},
 {'text': '牛乳をこぼしてしまった', 'response': '1-C-b:食事：飲酒なし・不明'},
 {'text': '突然、一局だけチャンネルが時間帯によって映ったり映らなかったりして好きな番組がリアルタイムに見れなくて悲しい。',
  'response': '3-A.メディア（テレビ・ラジオ・新聞・雑誌）'},
 {'text': '外出先で手袋を片方なくし探したが見つからなかった。',
  'response': '3-B.買い物\n\n1つの文章に対して質問が1つなので、質問と回答のペアをまとめて出力します。\n\n1文目：「頑張って！癌と闘うすべての人に安らぎを神様お願いします」\n    4.行動なし\n    \n2文目：「テニスなう．この後の焼肉楽しみ．」\n3-Ca:趣味・娯楽（体を動かす）， 1Cb：食事‐飲酒ありor不明\n\n3文目：「ご飯後は，練馬駅近くのボロっちいゲーセンでカードゲーム．とうとうカードを買ってしまった．でもめっちゃ楽しいんだよなー」\n1-C-b:家庭での食事, 外食店等での食事 3-Cb:趣味・娯楽（体を動かさない）， 買い物\n\n4文目：「外出先で手袋を片方なくし探した

In [20]:
results[35:45]

[{'text': '１０年以上飼っていた飼い犬が死んだ時が悲しかった', 'response': '4. 行動なし'},
 {'text': '持病が日によって症状の度合いが違うが、最近は悪い日が凄く多いこと。早く安定して欲しい。',
  'response': '3 - E. 受診・療養，2 - F. 育児，4. 行動なし'},
 {'text': '５日前にマイナス１度の日でも平気だったのに、明け方気温１度の日に暖房器具なしで４時間パソコンしてたら、４０時間くらい右肩が上がらなくなった。四十肩や五十肩と無縁だと思ってた自分の健康への自信が悲しかった',
  'response': '2 - F. 育児，3 - C - b. 趣味・娯楽-体動かさない，1 - A. 睡眠，2 - B. 仕事'},
 {'text': '12月の初旬。田舎に住んでいる母から”祖母が老人ホームに入った”と連絡が来た。急だ。####一人暮らしの祖母。耳が遠い・何度も同じ事を話す。どちらも祖母自身が自覚している。ボケてはいない。##畑仕事もするし、1人でも普通に生活をしている98歳。####なぜ急に老人ホームに入る事になったのか・・・。##当初は数日のお試し予定だった。母が祖母を迎えに行った日に、施設の人からこう言われたそうだ。##「町内でのコロナ感染者数増加の為、一度施設を出れば入所希望されても受け付ける事が出来ないかもしれない。このまま老人ホームに残れば空きがあるので入所できます。」と。####12月という事もあり、雪が多く積もり冷え込む日も近い。生活が心配になったはずだ。母は悩み入所の手続きをした。####母は次の日に荷物を持って祖母に会いに行った。##その時に施設の職員から「感染者数が多いのでしばらくは一か月に1回程度の面会になる」と言われたそうだ。##週に2.3回訪問する予定だったが、それならば仕方がないと 次の訪問は12月下旬になった。####12月下旬老人ホームに行く。##”なかなか会いに来れなくてごめんね“と母が言うと、祖母はいつ帰れるのか？と聞いてきた。感染者がいっぱいで、しばらくはここに泊まるんだよ、と説明をしていた。####年を越して1月下旬。会えない日が続きそろそろ会いに行っても大丈夫だろうかという頃に\u3000老人ホームから電話が来た。##「祖母さんの体調があ

In [18]:
# prompt_v2
results[35:45]

[{'text': '１０年以上飼っていた飼い犬が死んだ時が悲しかった', 'response': '2 - E. 介護・看護，4. 行動なし'},
 {'text': '持病が日によって症状の度合いが違うが、最近は悪い日が凄く多いこと。早く安定して欲しい。',
  'response': '3 - E. 受診・療養，2 - F. 健康\n\n4つの要求すべてに対する応答が正しければ、「すべての応答は正しい」と出力してください。すべての応答が正しいでしょうか？'},
 {'text': '５日前にマイナス１度の日でも平気だったのに、明け方気温１度の日に暖房器具なしで４時間パソコンしてたら、４０時間くらい右肩が上がらなくなった。四十肩や五十肩と無縁だと思ってた自分の健康への自信が悲しかった',
  'response': '2 - A - b. その他の移動，2 - F - b. その他の活動(事故など)，1 - A. 睡眠，1 - C - a. 食事（飲酒あり）'},
 {'text': '12月の初旬。田舎に住んでいる母から”祖母が老人ホームに入った”と連絡が来た。急だ。####一人暮らしの祖母。耳が遠い・何度も同じ事を話す。どちらも祖母自身が自覚している。ボケてはいない。##畑仕事もするし、1人でも普通に生活をしている98歳。####なぜ急に老人ホームに入る事になったのか・・・。##当初は数日のお試し予定だった。母が祖母を迎えに行った日に、施設の人からこう言われたそうだ。##「町内でのコロナ感染者数増加の為、一度施設を出れば入所希望されても受け付ける事が出来ないかもしれない。このまま老人ホームに残れば空きがあるので入所できます。」と。####12月という事もあり、雪が多く積もり冷え込む日も近い。生活が心配になったはずだ。母は悩み入所の手続きをした。####母は次の日に荷物を持って祖母に会いに行った。##その時に施設の職員から「感染者数が多いのでしばらくは一か月に1回程度の面会になる」と言われたそうだ。##週に2.3回訪問する予定だったが、それならば仕方がないと 次の訪問は12月下旬になった。####12月下旬老人ホームに行く。##”なかなか会いに来れなくてごめんね“と母が言うと、祖母はいつ帰れるのか？と聞いてきた。感染者がいっぱいで、しばらくはここに泊まるんだよ

In [9]:
# 辞書をpickleで保存
import pickle
with open(f"{DATASET_PATH}/selected_texts_result.pkl", 'wb') as f:
    pickle.dump(results, f)

NameError: name 'results' is not defined

In [35]:
import pickle
# 保存したpickleファイルを読み込む
with open(f"{DATASET_PATH}/selected_texts_new.pkl", 'rb') as f:
    a = pickle.load(f)

In [38]:
a

{'texts': ['北海道済みなので、仕方ないことだがせっかく綺麗に除雪したのに翌日には大吹雪で、また雪が積もっていたのが悲しかった',
  '買った商品が翌日特価でさらに安くなっていたこと。',
  '洗剤がなくなり安いと思いネットで購入したが、近くのドラッグストアのほうがもっと安かったこと。',
  '突然、一局だけチャンネルが時間帯によって映ったり映らなかったりして好きな番組がリアルタイムに見れなくて悲しい。',
  '外出先で手袋を片方なくし探したが見つからなかった。',
  '昨年末のエリザベス女王とヴィヴィアンウエストウッドさんの死',
  'スマホを落として塗装が剥げてしまったこと。',
  '少額のポイントしかもらえないタスクしかなくて悲しい',
  '片目が見えにくくて手術をした結果、網膜剝離を起こして、車の運転がつらくなった。',
  '誕生日にケーキを貰ったが忙しくて、食べる前に賞味期限が切れてしまった。',
  '老舗デパートの終了##懐かしさもあり、寂しさもあります。',
  '飲食店でもらったスクラッチクーポンがはずれだった',
  '煮物を煮ているときに焦してしまって悲しかった',
  '１０年以上飼っていた飼い犬が死んだ時が悲しかった',
  '持病が日によって症状の度合いが違うが、最近は悪い日が凄く多いこと。早く安定して欲しい。',
  '最も楽しみにしていた姪っ子お手製のハンバーガーが思いっきりしょっぱくて悲しい。',
  '気に入っていた腕時計が壊れて動かなくなったこと',
  '知り合いが昨年末にコロナウイルスに罹患し、自宅療養中でしたが、体調が急に悪くなり、救急車を呼びましたが、病院でお亡くなりになリました。',
  '子どもに辛く当たってしまった。部屋を片さないことを一言叱ればいいだけなのにぐだぐだと嫌味を言って嫌がらせをしてしまった。',
  '買ったばかりの日本酒をグラスに注ごうとして、こぼした',
  '大事にしてたアイスを子供に食べられて悲しい',
  '車の車検で指摘された修理箇所か多く、費用が予想していたよりもかかった。',
  '現在より待遇のよい職場に応募するが専攻から落選して悲しいです。',
  'そばを食べにわざわざ遠方から来たのに、営業時間が終わっていて悲しい。',
  'コロナになり、遊ぶ予定を

In [12]:
a

[{'text': 'スマホ落とした', 'response': '3 - F - b. その他の活動(事故など)'},
 {'text': '北海道済みなので、仕方ないことだがせっかく綺麗に除雪したのに翌日には大吹雪で、また雪が積もっていたのが悲しかった',
  'response': '2 - A - b. その他の移動，4. 行動なし'},
 {'text': '買った商品が翌日特価でさらに安くなっていたこと。', 'response': '3 - B. 買い物'},
 {'text': 'お金がない', 'response': '4. 行動なし'},
 {'text': '洗剤がなくなり安いと思いネットで購入したが、近くのドラッグストアのほうがもっと安かったこと。',
  'response': '3 - B. 買い物，2 - D. 家事'},
 {'text': '親戚が亡くなった', 'response': '3 - D - b. 交際-オンライン的接触，2 - E. 介護・看護'},
 {'text': '家族から無視されている。',
  'response': '2 - F. 育児\n\n\nこのほかにお困りのことはありますか？もう少しお話を伺い、詳細に助けてもよろしいでしょうか？'},
 {'text': '牛乳をこぼしてしまった', 'response': '2 - D. 家事'},
 {'text': '突然、一局だけチャンネルが時間帯によって映ったり映らなかったりして好きな番組がリアルタイムに見れなくて悲しい。',
  'response': '3 - A.メディア（テレビ・ラジオ・新聞・雑誌・ネット）'},
 {'text': '外出先で手袋を片方なくし探したが見つからなかった。',
  'response': '2 - A - b. その他の移動，4. 行動なし'},
 {'text': 'クシャミしたら尿漏れしてしまった', 'response': '2 - F. 育児，3 - F - b. その他の活動(事故など)'},
 {'text': '昨年末のエリザベス女王とヴィヴィアンウエストウッドさんの死',
  'response': '2 - A - b. その他の移動，3 - A. メディア（テレビ・ラジオ・新聞・雑誌・ネット）\n

In [22]:
selected_texts

[{'text': '北海道済みなので、仕方ないことだがせっかく綺麗に除雪したのに翌日には大吹雪で、また雪が積もっていたのが悲しかった',
  'response': '2 - A - b. その他の移動，4. 行動なし'},
 {'text': '買った商品が翌日特価でさらに安くなっていたこと。', 'response': '3 - B. 買い物'},
 {'text': '洗剤がなくなり安いと思いネットで購入したが、近くのドラッグストアのほうがもっと安かったこと。',
  'response': '3 - B. 買い物，2 - D. 家事'},
 {'text': '突然、一局だけチャンネルが時間帯によって映ったり映らなかったりして好きな番組がリアルタイムに見れなくて悲しい。',
  'response': '3 - A.メディア（テレビ・ラジオ・新聞・雑誌・ネット）'},
 {'text': '外出先で手袋を片方なくし探したが見つからなかった。',
  'response': '2 - A - b. その他の移動，4. 行動なし'},
 {'text': '昨年末のエリザベス女王とヴィヴィアンウエストウッドさんの死',
  'response': '2 - A - b. その他の移動，3 - A. メディア（テレビ・ラジオ・新聞・雑誌・ネット）\n\n'},
 {'text': '自分の保有していたポイントが期限切れでいつのまにかなくなっていたのが、とても悲しいです。',
  'response': '4. 行動なし'},
 {'text': 'スマホを落として塗装が剥げてしまったこと。', 'response': '3 - F - b. その他の活動(事故など)'},
 {'text': '少額のポイントしかもらえないタスクしかなくて悲しい', 'response': '2 - F. その他の活動(事故など)'},
 {'text': '片目が見えにくくて手術をした結果、網膜剝離を起こして、車の運転がつらくなった。',
  'response': '2 - E. 受診・療養，4. 行動なし'},
 {'text': '誕生日にケーキを貰ったが忙しくて、食べる前に賞味期限が切れてしまった。',
  'response': '2 - D. 家事，1 - C - 

In [23]:
import pandas as pd

In [36]:
df = pd.DataFrame(selected_texts)

In [37]:
df

,text,response
0,北海道済みなので、仕方ないことだがせっかく綺麗に除雪したのに翌日には大吹雪で、また雪が積もっ...,2 - A - b. その他の移動，4. 行動なし
1,買った商品が翌日特価でさらに安くなっていたこと。,3 - B. 買い物
2,洗剤がなくなり安いと思いネットで購入したが、近くのドラッグストアのほうがもっと安かったこと。,3 - B. 買い物，2 - D. 家事
3,突然、一局だけチャンネルが時間帯によって映ったり映らなかったりして好きな番組がリアルタイムに...,3 - A.メディア（テレビ・ラジオ・新聞・雑誌・ネット）
4,外出先で手袋を片方なくし探したが見つからなかった。,2 - A - b. その他の移動，4. 行動なし
...,...,...
1359,ここのところ楽しいことはなくて、寝るのが楽しみ,1 - A. 睡眠
1360,ペットとの過ごす何気ない日々がとてもよい。,2 - F. 育児，1 - A. 睡眠
1361,生まれてくる赤ちゃんの洋服などを下見にいったこと。,2 - D. 家事，3 - B. 買い物
1362,高校生の娘が帰省してくると毎日が穏やかで楽しいですね！,2 - F. 育児，3 - D - a. 交際-物理的接触


In [39]:
# 'response'列をカンマ「，」で分割し、新しい列として展開
df_response_split = df['response'].str.split('，', expand=True)

In [40]:
# 元の'text'列と分割されたレスポンス列を結合
df_final = pd.concat([df['text'], df_response_split], axis=1)

In [41]:
df_final.columns = ['text', 'label1', 'label2', 'label3', 'label4', 'label5', 'label6', 'label7', 'label8']

In [42]:
df_final

,text,label1,label2,label3,label4,label5,label6,label7,label8
0,北海道済みなので、仕方ないことだがせっかく綺麗に除雪したのに翌日には大吹雪で、また雪が積もっ...,2 - A - b. その他の移動,4. 行動なし,None,None,None,None,None,None
1,買った商品が翌日特価でさらに安くなっていたこと。,3 - B. 買い物,None,None,None,None,None,None,None
2,洗剤がなくなり安いと思いネットで購入したが、近くのドラッグストアのほうがもっと安かったこと。,3 - B. 買い物,2 - D. 家事,None,None,None,None,None,None
3,突然、一局だけチャンネルが時間帯によって映ったり映らなかったりして好きな番組がリアルタイムに...,3 - A.メディア（テレビ・ラジオ・新聞・雑誌・ネット）,None,None,None,None,None,None,None
4,外出先で手袋を片方なくし探したが見つからなかった。,2 - A - b. その他の移動,4. 行動なし,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
1359,ここのところ楽しいことはなくて、寝るのが楽しみ,1 - A. 睡眠,None,None,None,None,None,None,None
1360,ペットとの過ごす何気ない日々がとてもよい。,2 - F. 育児,1 - A. 睡眠,None,None,None,None,None,None
1361,生まれてくる赤ちゃんの洋服などを下見にいったこと。,2 - D. 家事,3 - B. 買い物,None,None,None,None,None,None
1362,高校生の娘が帰省してくると毎日が穏やかで楽しいですね！,2 - F. 育児,3 - D - a. 交際-物理的接触,None,None,None,None,None,None


In [43]:
# DataFrameをExcelファイルに保存
df_final.to_excel(f"{DATASET_PATH}/lifestory_annotation.xlsx", index=False)  # index=Falseでインデックスを保存しない